In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/ClassAug Work/final code"

/content/drive/.shortcut-targets-by-id/12PFZZk88jXqRuPPPvsCgycke0gd7aQBI/ClassAug Work/final code


In [ ]:
import tensorflow as tf
from tensorflow import keras
import pickle
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, average_precision_score, precision_recall_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA

In [ ]:
data = None
with open('./embeddings/twitter_us_airline_sentiment_embeddings.pkl', 'rb') as f:
    data = pickle.load(f)
    
embs = data['embs']
labels = data['labels']

In [ ]:
label_mapping_reverse = {
    0: 'neutral',
    1: 'positive',
    2: 'negative'
}

label_mapping = {}
for k, v in label_mapping_reverse.items():
    label_mapping[v] = k

In [ ]:
classes = list(label_mapping.keys())

adhoc_classes = list(combinations(classes, 2))
adhoc_classes = ['|'.join(comb) for comb in adhoc_classes]

all_classes = classes
all_classes.extend(adhoc_classes)

for i, c in enumerate(all_classes):
    if c not in label_mapping:
        label_mapping[c] = i

for i, c in enumerate(all_classes):
    if i not in label_mapping_reverse:
        label_mapping_reverse[i] = c

In [ ]:
crs = np.array([tuple(map(lambda x: int(label_mapping[x]), adhoc_class.split("|"))) for adhoc_class in adhoc_classes])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(embs, labels, test_size=0.30, random_state=43, stratify=labels)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(10248, 512)
(4392, 512)
(10248,)
(4392,)


In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, embs, labels, batch_size=64, dim=512, n_classes=6, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.embs = embs
        self.n_classes = n_classes
        self.labels = tf.keras.utils.to_categorical(labels, num_classes=self.n_classes, dtype=np.int16)
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.embs.shape[0] / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(self.embs.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):        
        X = self.embs[indexes]
        y = self.labels[indexes]

        p = np.random.permutation(np.arange(X.shape[0]))
        X = X[p]
        y = y[p]
        
        return X, y

In [ ]:
class AdhocDataGenerator(keras.utils.Sequence):
    def __init__(self, embs, labels, label_mapping, label_mapping_reverse, batch_size=64, adhoc_batch_size=192, dim=512, n_classes=3, n_adhoc_classes=3, crs=[], shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.adhoc_batch_size = adhoc_batch_size
        self.total_batch_size = batch_size + adhoc_batch_size
        self.label_mapping = label_mapping
        self.label_mapping_reverse = label_mapping_reverse
        self.embs = embs
        self.n_classes = n_classes
        self.n_adhoc_classes = n_adhoc_classes
        self.crs = crs
        self.total_classes = n_classes + n_adhoc_classes
        self.one_cold_labels = np.squeeze(labels)
        self.labels = keras.utils.to_categorical(labels, num_classes=self.total_classes, dtype=np.int16)
        self.shuffle = shuffle
        self.on_epoch_end()
        self.__generate_confused_examples()
       
    def __generate_confused_examples(self):
        idx = np.random.choice(len(self.crs), size=self.embs.shape[0])
        class1, class2 = self.crs[idx][:,0], self.crs[idx][:,1]
        confused_examples = np.zeros((self.embs.shape[0], 2, self.dim))
        confused_examples_labels = np.zeros((self.embs.shape[0], self.total_classes))
        class_examples = {}
        i = 0
        for c1, c2 in zip(class1, class2):
            if i % 10000 == 0:
                print(f'Examples #{i}/{class1.shape[0]}')
            if c1 not in class_examples:
                class_examples[c1] = self.embs[self.one_cold_labels == c1]
            if c2 not in class_examples:
                class_examples[c2] = self.embs[self.one_cold_labels == c2]
            
            confused_examples[i][0] = self.__get_random_choice(class_examples[c1]) + np.random.normal(0.5, 1, self.dim)
            confused_examples[i][1] = self.__get_random_choice(class_examples[c2]) + np.random.normal(0.5, 1, self.dim)
            
            confused_examples_labels[i] = keras.utils.to_categorical(
                self.label_mapping[
                    self.label_mapping_reverse[c1] + '|' + self.label_mapping_reverse[c2]
                ]
            , num_classes=self.total_classes)
            
            i += 1
        class_examples = {}
        lamda = np.random.uniform(0.4, 0.6, size=self.embs.shape[0]).reshape(-1, 1)
        
        self.confused_examples = (lamda*confused_examples[:,0]) + ((1-lamda)*confused_examples[:,1])
        self.confused_examples_labels = confused_examples_labels

    def __len__(self):
        return int(np.floor(self.embs.shape[0] / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(self.embs.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __get_random_choice(self, arr):
        idx = np.random.choice(len(arr))
        return arr[idx]
            
    def __data_generation(self, indexes):
        batch_x = self.embs[indexes]
        batch_y = self.labels[indexes]
        
        X = np.zeros(([self.batch_size+self.adhoc_batch_size, self.dim]), dtype=np.float32)
        y = np.zeros(([self.batch_size+self.adhoc_batch_size, self.total_classes]), dtype=np.int16)
        
        X[:self.batch_size, :] = self.embs[indexes]
        y[:self.batch_size, :] = self.labels[indexes]
        
        idx = np.random.choice(self.confused_examples.shape[0], size=self.adhoc_batch_size)
        X[self.batch_size:, :] = self.confused_examples[idx]
        y[self.batch_size:, :] = self.confused_examples_labels[idx]
        
        p = np.random.permutation(np.arange(X.shape[0]))
        X = X[p]
        y = y[p]
        
        return X, y

In [ ]:
training_generator = AdhocDataGenerator(x_train, y_train, label_mapping, label_mapping_reverse, crs=crs)
validation_generator = DataGenerator(x_test, y_test)

Examples #0/10248
Examples #10000/10248


In [ ]:
keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, activity_regularizer='l2', activation='relu', input_shape=(512,)))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(3+3, activity_regularizer='l2', activation='relu'))
model.add(keras.layers.Softmax())

In [ ]:
# model.load_weights('./models/')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 3078      
                                                                 
 softmax (Softmax)           (None, 6)                 0         
                                                                 
Total params: 265,734
Trainable params: 265,734
Non-trainable params: 0
_________________________________________________________________


In [ ]:
checkpoint_filepath = './checkpoints/classaug_twitter_us_airline_sentiment_best_{val_accuracy:.2f}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
filepath=checkpoint_filepath,
save_weights_only=True,
monitor='val_accuracy',
mode='max',
save_best_only=True)
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.1), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
history=model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[model_checkpoint_callback])

Epoch 1/1000
160/160 [==============================] - 2s 9ms/step - loss: 1.5793 - accuracy: 0.4339 - val_loss: 1.2414 - val_accuracy: 0.6271
Epoch 2/1000
160/160 [==============================] - 1s 7ms/step - loss: 1.3808 - accuracy: 0.4807 - val_loss: 1.1671 - val_accuracy: 0.6261
Epoch 3/1000
160/160 [==============================] - 1s 5ms/step - loss: 1.3229 - accuracy: 0.5236 - val_loss: 1.1151 - val_accuracy: 0.6266
Epoch 4/1000
160/160 [==============================] - 1s 5ms/step - loss: 1.2740 - accuracy: 0.5611 - val_loss: 1.0831 - val_accuracy: 0.6268
Epoch 5/1000
160/160 [==============================] - 1s 5ms/step - loss: 1.2193 - accuracy: 0.6001 - val_loss: 1.0514 - val_accuracy: 0.6278
Epoch 6/1000
160/160 [==============================] - 1s 5ms/step - loss: 1.1565 - accuracy: 0.6416 - val_loss: 1.0212 - val_accuracy: 0.6356
Epoch 7/1000
160/160 [==============================] - 1s 5ms/step - loss: 1.0844 - accuracy: 0.6910 - val_loss: 0.9939 - val_accuracy: